
---
# Unit03: Processing passive seismic data

This notebook has some practical processing activities of the Course **ProSeisSN**. It deals with data processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib

#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

In [5]:
# Import Libraries
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import rcParams
"""
Activate the ipympl backend by including
%matplotlib ipympl
magic in the notebook or by
%matplotlib widget
which will have the same effect.

Other magic:
%matplotlib inline
%matplotlib notebook
"""
%matplotlib widget

# Import specialized ObsPy packages + sanity.
try:
    import obspy
    print('obspy version==>', obspy.__version__)
except TypeError:
    print('Stopping RUNTIME. ObsPy not found.')
    exit()

from obspy import read
from obspy import Stream
from obspy import UTCDateTime
from obspy import read, Stream
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.header import FDSNException

obspy version==> 1.2.2


---
## Accessing data

We work with data form an event occured in Acre in 2024,

|EventID | Time | Lat | Long | Depth(km) | Mww | Location |
| :- | :-: | :- | :- | :-: | :-: | :- |
|11793085|2024-01-20T21:31|-7.2879|-71.464|607|6.6|WESTERN BRAZIL|

As recorded at station IU HKT, distant $44^{\circ}$ from the event. We have two options:

- **Read** the data locally $\Longrightarrow$ skip next code, or

- **Retrive** the data from client **IRIS** $\Longrightarrow$ run next code skipping the next one.

In [ ]:
"""
=========== ONLY RUN THIS CODE TO RETRIEVE THE DATA FROM IRIS. SKIP THE NEXT CODE =========== 
"""
# Specify event's start and end times
starttime = UTCDateTime("2024-01-20T21:37:19.019539Z") - 60
endtime = UTCDateTime("2024-01-20T21:58:18.994539Z")
print(starttime, endtime)

# Use wildcards to select all three HH* channels
net = "IU"
sta = "HKT"
loc = "00"
chan = "HH*"

# Specify client. Opt:from obspy.clients.earthworm import Client
# Get waveforms with instrument response into a stream
# List of ObsPy clients: clients = ["IRIS", "NCEDC", "USGS", "GEONET", "RESIF", "INGV", "BGR", "ODC", "SCEDC"]
try:
  client = Client("IRIS")
  st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)
except FDSNException:
  print(f"Chosen client is not working.")
  from obspy.clients.earthworm import Client
  client = Client("ETH")
  st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)

# Remove the instrument response in all the 3 components/traces.
st.remove_response(output = 'VEL')

In [3]:
"""
=========== ONLY RUN THIS CODE TO READ THE DATA LOCALLY ===========
"""
# Read the data into a stream object
st = read('wb11793085r.msd')

Stream of Acre event at station IU HKT:
 3 Trace(s) in Stream:
IU.HKT.00.HH1 | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:14.538394Z | 100.0 Hz, 41552 samples
IU.HKT.00.HH2 | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:17.648394Z | 100.0 Hz, 41863 samples
IU.HKT.00.HHZ | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:17.688394Z | 100.0 Hz, 41867 samples


In [4]:
# Print stream information
print(f"Stream of Acre event at station IU HKT:\n {st}")
print(f"Characteristis for the Z-component:\n {st[-1].stats}")

Stream of Acre event at station IU HKT:
 3 Trace(s) in Stream:
IU.HKT.00.HH1 | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:14.538394Z | 100.0 Hz, 41552 samples
IU.HKT.00.HH2 | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:17.648394Z | 100.0 Hz, 41863 samples
IU.HKT.00.HHZ | 2024-01-20T21:36:19.028394Z - 2024-01-20T21:43:17.688394Z | 100.0 Hz, 41867 samples
Z-component characteristis:
          network: IU
         station: HKT
        location: 00
         channel: HHZ
       starttime: 2024-01-20T21:36:19.028394Z
         endtime: 2024-01-20T21:43:17.688394Z
   sampling_rate: 100.0
           delta: 0.01
            npts: 41867
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 748, 'encoding': 'FLOAT64', 'byteorder': '>', 'record_length': 512, 'filesize': 1145856})


# ==> My memos from Volcanoseismology <== to be deleted!

- lecture_14_theory_fourier.ipynb

- lecture_15_theory_filter.ipynb

- lecture_16_theory_spectral_analysis_preprocessing.ipynb

- lecture_18_obspy_signal_processing.ipynb

- lecture_19_obspy_instrument_response.ipynb
